# 🚢 Ship Happens

## 1️⃣ Intro
### 🚚🧦🚀 The Great SoloSock Shipping Challenge

Welcome, brave SP-API developer!

Our hero, Harsh, runs SoloSock, the hottest new mismatched sock business. Sales are soaring, but there’s two problems:

His printer only supports PNG label formats. No ZPL. No PDF. Just pure, pixel-perfect PNG.

_And guess what?_

Today, Chris “The Fulfillment Guru” Khoury placed an urgent order from Debugville. Harsh needs to get that order delivered ASAP. Only the fastest shipping rate will do!

But his Buy Shipping script is broken! 💀

Harsh needs your help. His Buy Shipping script is broken, and he can’t ship SoloSocks.


Fix the code below so the label is generated successfully.


### 🛠️ Your Mission

1. Get Rates – Fix the script so it pulls available shipping rates.

2. Pick the Fastest Rate – From the list of rates, choose the one with the earliest delivery promise.

3. Ensure PNG Format – The printer can only use labels in PNG format.

4. Fix the Purchase Request – Plug in the right rateId and format details to get the label generated.

5. Celebrate like a Fulfillment Wizard.

## 2️⃣ Setup
### 🧰 Installing Selling Partner API SDK

The **Amazon Selling Partner API SDK** is an official Amazon library that simplifies integration with Amazon’s SP-API. It handles things like authentication, request signing, and response parsing — so you can focus on building your solution instead of dealing with low-level API mechanics.

With this SDK, you can easily access endpoints like Orders, Listings, Reports, Feeds, and **Buy Shipping** using clean, structured Python code.

📚 Learn more or explore the source on GitHub: https://github.com/amzn/selling-partner-api-sdk/tree/main


📦 To install the SDK, run the below block 👇




In [ ]:
pip install amzn-sp-api

---

## 3️⃣ The Shipping Challenge
### 🐛 Mocking the SP-API server response

Harsh doesn’t have real API credentials — but luckily, the Amazon SP API team built a mock server just for this scenario.

Below you will find `mock_oauth_endpoint` and `mock_endpoint` variables. Use the endpoints below to test without real tokens:

In [ ]:
# Configuration for mock SP-API
mock_oauth_endpoint = "http://44.246.246.165:80/auth/o2/token"
mock_endpoint = "http://44.246.246.165:80"
print("MockEndpoint: " + mock_endpoint)

Now let's configure the SDK:

In [ ]:
from spapi import SPAPIConfig, SPAPIClient, ApiException, ShippingApi
from spapi.models.shipping_v2 import GetRatesResponse, PurchaseShipmentResponse

config = SPAPIConfig(
    client_id="Harsh",
    client_secret="Socker",
    refresh_token="FakeToken",
    region="NA"
)
client = SPAPIClient(config, endpoint=mock_endpoint, oauth_endpoint=mock_oauth_endpoint)
print("Succesfully created SP-API client")

---

### 📦 Step 0: Let’s get the order information. You need to create the shipment for orderId 111-1111111-1111111

In [ ]:
from spapi import OrdersV0Api
from spapi.models.orders_v0 import GetOrdersResponse

order_api = OrdersV0Api(client.api_client)
print("Succesfully created order API")
#TODO here
get_order_response = order_api.get_order()
print(get_order_response.payload)

### 📦 Step 1: Let’s fetch the available rates!

In [ ]:
shipping_api = ShippingApi(client.api_client)
print("Succesfully created shipping API")

The first thing is get a list of available shipping options. He knows where he’s shipping from (SoloSock HQ) and to (Chris in Debugville), so we’ll use that information to call the getRates endpoint.

This step is like going to the post office and asking, “Hey, how what options and how much will it cost to ship this package, and how fast can you get it there?”

Use the code below to structure the request:

In [ ]:
def get_rates_request():
    return {
        "shipFrom": {
            "name": "Harsh The Sock Hero",
            "addressLine1": "Laundry Room B2",
            "city": "Seattle",
            "stateOrRegion": "IM",
            "postalCode": "40404",
            "countryCode": "US",
            "companyName": "SoloSock Inc.",
            "phoneNumber": "+1-867-5309"
        },
        "shipTo": {
            "name": "Chris 'Fulfillment Guru' Khoury",
            "addressLine1": "Amazon API Temple",
            "addressLine2": "Suite 404 – Socks Not Found",
            "city": "Debugville",
            "stateOrRegion": "WA",
            "postalCode": "98199",
            "countryCode": "US"
        },
        "packages": [
            {
                "dimensions": {"length": 6, "width": 4, "height": 2, "unit": "INCH"},
                "weight": {"unit": "GRAM", "value": 120},
                "items": [{
                    "itemIdentifier": "111111111111111",
                    "quantity": 1,
                    "itemValue": {"unit": "USD", "value": "7.99"},
                    "weight": {"unit": "GRAM", "value": 120}
                }],
                "insuredValue": {"unit": "USD", "value": "7.99"},
                "packageClientReferenceId": "Order_111-1111111-1111111_Package"
            }
        ],
        "channelDetails": {
            "channelType": "AMAZON",
            "amazonOrderDetails": {"orderId": "111-1111111-1111111"}
        }
    }

---

Once the request is structured, you’ll call the get_rates() function to fetch those shipping options: Here you may get error saying that you missing some attributes. Please feel the missing attributes correctly

In [ ]:
get_rates_response = GetRatesResponse(shipping_api.get_rates(body=get_rates_request()).payload)
request_token = get_rates_response.payload.request_token
print(request_token)

🧠 The `request_token` is important — it ties the rates to your purchase and will be used for make purchasement at later steps

---

### 🕵️  Step 2: Let’s inspect what’s available!

Now that we’ve got a list of potential shipping options, it’s time to analyze them. We’ll print out details for each rate, including:

- The rate ID
- The carrier
- The total cost
- The available label formats
- The estimated delivery window

This is where Harsh wants your help to pick the fastest rate — the one that delivers the soonest.

Run this function to view them:

In [ ]:
def print_returned_rates(rates, full_response=False):
    print("Available Shipment Rates:")
    if full_response:
        print("getRates Full Response Payload:", rates)
    else:
        for i, rate in enumerate(rates):
            for doc_format in rate.supported_document_specifications:
                print(f"[{i}] rateId: {rate.rate_id} "
                      f"-- carrierName: {rate.carrier_id} "
                      f"-- totalCharge: {rate.total_charge.value} "
                      f"-- availableFormat: {doc_format.format} "
                      f"-- deliveryWindow: {rate.promise}")

print_returned_rates(get_rates_response.payload.rates, full_response=False)

You’ll see the index in square brackets []. That will come in handy for the next step.

🔍 Pro tip: Make sure to pick a rate that supports PNG format. Harsh’s printer doesn’t speak ZPL.

---

### 🕵️  Step 3: Time to choose your rate!

From the list of available shipping options, it’s your job to select the fastest rate that also supports the PNG label format. You can do this manually by checking the deliveryWindow and the format printed earlier.

Once you know which one to pick, set it like this:

In [ ]:
# TODO here
idx = 
selected_rate = get_rates_response.payload.rates[idx]

⚠️ Make sure the selected rate supports PNG — or the final label won’t be printable!

---

### 🧾 Step 4: Make the purchase and get that label!

You’ve selected the rate. You’ve got the request token. Now it’s time to purchase the shipment and retrieve the shipping label in PNG format.

This is where you take all the magical ingredients you've gathered:

- The selected shipping rateId
- The label format (Harsh’s printer is allergic to anything but PNG)

You'll combine them all into one beautifully structured request. If you mess this part up, your label might come back in ZPL and Harsh’s printer will scream. 😱

So double-check the format, make sure it was listed in the supported formats from getRates. This is the final step before your label appears. You got this. 🧦📦

In [ ]:
def extract_label_details(rate_model):
    size = None
    option = None

    for spec in rate_model.supported_document_specifications:
        if spec.format == "PNG":
            size = spec.size
            option = spec.print_options[0]
    return size, option

def purchase_shipment_request(rate, token):
    label_size, print_options = extract_label_details(rate)

    return {
        "rateId": rate.rate_id,
        "requestedDocumentSpecification": {
            "format": "PNG",
            "size": label_size,
            "needFileJoining": print_options,
            "requestedDocumentTypes": print_options,
            "pageLayout": print_options
        },
        "requestedValueAddedServices": [
            {"id": rate.available_value_added_service_groups[0].value_added_services[0].id}
        ],
        "requestToken": token
    }

---

You've selected the fastest rate. You've crafted the perfect purchase request. Now it's time to send it off to SP-API and retrieve the PNG shipping label that Harsh’s printer will actually accept.

Run the code below to purchase the shipment using the request you just built. If all goes well, you’ll see:

- A shiny shipmentId
- A PNG label document
- And a very relieved sock founder

😬 Got an error? Don't panic. It probably means something in your request was off — maybe the format wasn’t supported or a value was missing.

✏️ Go back, tweak the `purchase_shipment_request()` function, and rerun both the request and the purchase blocks.

In [ ]:
 # Purchasing Shipment
#print(purchase_shipment_request(rate=selected_rate, token=request_token))
purchase_response = shipping_api.purchase_shipment(body=purchase_shipment_request(rate=selected_rate, token=request_token))
print(purchase_response)

#shipment = purchase_response.payload
# print("📦 Shipment ID:", shipment.shipment_id)

# for detail in shipment.package_document_details:
#     for doc in detail.package_documents:
#         print("🧾 Document Format:", doc.format)
#         print("📄 Document Contents:", doc.contents)

---

## 🏁 Challenge Complete!

🎉 You did it! You fixed Harsh’s broken Buy Shipping flow and got a working PNG label. The socks are en route, and Debugville is saved.

You’ve now used:

- get_rates() to fetch shipping options
- Parsed and selected the best rate
- Ensured label compatibility with Harsh’s printer
- Called purchase_shipment() with correct parameters
- Printed out your beautiful label response